J'essaie de scraper des infos sur la page de présentation Fragantica de Black Phantom : https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html

Je veux scraper ces infos :
Nom du Parfum
Nom de la marque
Launch year
Perfume rating
Nombre de votant sur le Perfume rating
Saisonnalité (Hiver, automne… peut etre gardé juste la + votée)
Parfumeur
Main accord
‘Perfume Pyramide’  = [‘Top note’ , ‘Middle note’, Base note’]
Genre (en prenant celui qui à le plus de vote)
Longevite, Sillage (en prenant les votes)


## Imports

In [60]:
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import cloudscraper

ModuleNotFoundError: No module named 'cloudscraper'

## Les fonctions

In [52]:
#Récupère le HTML de la page correspondant à l'URL donnée.
#Le driver doit déjà être initialisé à l'extérieur de la fonction.
def get_html(driver, url):
    driver.get(url)
    return driver.page_source

## Le script

In [ ]:
#1 CONFIGARATION du DRIVER CHROME de SELENIUM

##On configure Chrome pour être en "headless" (sans interface)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
##On crée le driver en précisant les options headless
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)


#2 BOUCLE DE LA FONCTION POUR CHAQUE URL
for url in urls:
    # 3. On récupère le HTML de la page ~~~~~~~~~~~~~~~~~~~~~~~~à chaque itération
    html = get_html(driver, url)
    soup = BeautifulSoup(html, 'html.parser')

    # 4. On extrait les informations de la page ~~~~~~~~~~~~~~~~à chaque itération
    #~~~~~~A ECRIRE

    # 5. On stocke les informations dans un fichier CSV ~~~~~~~à chaque itération
#~~~~~~A ECRIRE


# 5. On ferme le driver ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~à la fin du script
driver.quit()


In [ ]:
#Fonction qui scrape les variable d'un parfum à partir de son code html

def scrape_fragrantica_perfume(html_content):
    """
    Scrap les informations principales d'une page Fragrantica
    Retourne un dictionnaire avec :
      - nom_parfum
      - marque
      - launch_year
      - perfume_rating
      - nb_votes
      - saison_plus_votee
      - parfumeur
      - main_accord
      - perfume_pyramid (dict avec top/middle/base)
      - genre_plus_vote
      - longevite_votes
      - sillage_votes
    """

    
#~~~~~~~~~~~~~~~~~~~~~~~~~~OK~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    data = {}

    # 1) Nom du parfum        
    match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', url)
    if match:
        perfume_name = match.group(1)
        perfume_name = perfume_name.replace('-', ' ')
        data['nom_parfum'] = perfume_name
    else:
        data['nom_parfum'] = None

    #2) Marque
    match = re.search(r'/perfume/([^/]+)/', url)
    if match:
        brand_name = match.group(1)
        brand_name = brand_name.replace('-', ' ')
        data['marque'] = brand_name
    else:
        data['marque'] = None

#~~~~~~~~~~~~~~~~~~~~~~~~~~OK~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        
    # 2) Launch year
    # Souvent indiqué dans le titre ou dans un <span class="perfume-year">2017</span> par exemple
    # On peut tenter de le récupérer directement dans le titre s'il n'y a pas d'élément dédié.
    
    launch_year = None
    # Essai via un selecteur direct (parfois .perfume-year ou .year)
    year_el = soup.select_one('.perfume-year')
    if year_el:
        # exemple : "2017"
        launch_year = year_el.get_text(strip=True)
    else:
        # Plan B : chercher un 4 chiffres dans le titre <h1> par ex.
        h1 = soup.select_one('h1')
        if h1:
            match_year = re.search(r'(\d{4})', h1.get_text())
            if match_year:
                launch_year = match_year.group(1)
    
    data['launch_year'] = launch_year
    
    # 3) Perfume rating & nombre de votes
    # Sur la page, c'est souvent dans un bloc "vote-box" : "4.54 out of 5 with 3270 votes"
    # On peut donc chercher un pattern  : ([0-9.]+) out of 5 with ([\d,]+) votes
    
    rating_block = soup.select_one('.vote-box')
    perfume_rating = None
    nb_votes = None
    
    if rating_block:
        text_block = rating_block.get_text(strip=True)
        # Exemple de texte : "4.54 out of 5 with 3270 votes"
        match = re.search(r'([\d.]+)\s*out of 5 with\s*([\d,]+)\s*votes', text_block)
        if match:
            perfume_rating = match.group(1)
            nb_votes = match.group(2).replace(',', '')
    
    data['perfume_rating'] = perfume_rating
    data['nb_votes'] = nb_votes
    
    # 4) Saisonnalité
    # On peut la voir dans un bloc "Seasons" ou un mini-histogramme. 
    # Sur la page Black Phantom, il y a une section "This perfume is best for" (Seasons).
    # Il faut parfois parser un diagramme ou un <div class="bars">. Ici, on cherche la saison qui a le plus de votes. 
    # On peut ajuster en fonction du HTML réel. 
    
    # Exemple hypothétique : 
    saison_plus_votee = None
    seasons_div = soup.select_one('#diagram_seasons')  # ID ou classe à adapter
    if seasons_div:
        # On cherche par ex. chaque <div class="bars"><span>Summer</span><span>180</span></div>
        # On va trouver la saison qui a le maximum de votes
        max_votes = 0
        for bar in seasons_div.select('.bars'):
            spans = bar.find_all('span')
            if len(spans) == 2:
                saison_name = spans[0].get_text(strip=True)
                saison_vote = spans[1].get_text(strip=True)
                try:
                    saison_vote = int(saison_vote)
                    if saison_vote > max_votes:
                        max_votes = saison_vote
                        saison_plus_votee = saison_name
                except ValueError:
                    pass
    
    data['saison_plus_votee'] = saison_plus_votee
    
    # 5) Parfumeur
    # Souvent mentionné dans un bloc : "Perfumer: ..."
    # On peut chercher un <span> ou <a> à côté d'un label 'Perfumer:'
    
    parfumeur = None
    # Stratégie : repérer un bloc qui contient "Perfumer:" et extraire la suite
    potential_perfumer = soup.find(text=re.compile(r'Perfumer:', re.IGNORECASE))
    if potential_perfumer:
        # le parent pourrait contenir le lien vers le parfumeur
        parent = potential_perfumer.parent
        # Chercher un <a> dans ce parent
        link = parent.find('a')
        if link:
            parfumeur = link.get_text(strip=True)
        else:
            # Sinon extraire le texte brut
            next_text = parent.get_text(strip=True)
            # exemple "Perfumer: Sidonie Lancesseur"
            # On peut faire un split sur ':'
            split_data = next_text.split(':', 1)
            if len(split_data) == 2:
                parfumeur = split_data[1].strip()
    
    data['parfumeur'] = parfumeur
    
    # 6) Main accord
    # Généralement listé dans la section "Main Accords" en haut, sous forme de blocs colorés
    # Sur Fragrantica, ça peut être un <div class="userMainAccords"> ... <span class="accord">Gourmand</span>
    # On récupère uniquement le premier (ou tous).
    
    main_accord = None
    user_main_accords = soup.select_one('.userMainAccords')
    if user_main_accords:
        first_accord = user_main_accords.select_one('.accord-box')
        if first_accord:
            main_accord = first_accord.get_text(strip=True)
    
    data['main_accord'] = main_accord
    
    # 7) Perfume Pyramid = top, middle, base notes
    # Sur la page, c'est la section "Perfume Pyramid" avec un ID #pyramid
    # On y trouve ensuite <div class="col">Top Notes</div>, etc.
    
    pyramid = soup.select_one('#pyramid')
    perfume_pyramid = {
        'top_notes': [],
        'middle_notes': [],
        'base_notes': []
    }
    if pyramid:
        # Rechercher les sections "Top Notes", "Middle Notes", "Base Notes"
        # Sur Fragrantica, c'est souvent un <h4> ou <div class="col">Top Notes</div>, puis des images <img alt="NoteName"/>
        
        # Ex top
        top_section = pyramid.find(lambda tag: tag.name == 'div' and 'Top Notes' in tag.get_text())
        if top_section:
            # Les notes peuvent être dans les balises <img alt="Coffee">, etc.
            top_imgs = top_section.find_next_sibling().select('img[alt]') if top_section.find_next_sibling() else []
            perfume_pyramid['top_notes'] = [img['alt'] for img in top_imgs]
        
        # Middle
        middle_section = pyramid.find(lambda tag: tag.name == 'div' and 'Middle Notes' in tag.get_text())
        if middle_section:
            middle_imgs = middle_section.find_next_sibling().select('img[alt]') if middle_section.find_next_sibling() else []
            perfume_pyramid['middle_notes'] = [img['alt'] for img in middle_imgs]
        
        # Base
        base_section = pyramid.find(lambda tag: tag.name == 'div' and 'Base Notes' in tag.get_text())
        if base_section:
            base_imgs = base_section.find_next_sibling().select('img[alt]') if base_section.find_next_sibling() else []
            perfume_pyramid['base_notes'] = [img['alt'] for img in base_imgs]

    data['perfume_pyramid'] = perfume_pyramid
    
    # 8) Genre (avec le plus de votes)
    # Souvent, c'est "for women and men", ou "for men", "for women". Parfois il y a un sondage "66% de votes: c’est un parfum mixte" etc.
    # On peut chercher dans un bloc "votes for unisex/men/women"
    
    genre_plus_vote = None
    genre_div = soup.select_one('#gender')
    if genre_div:
        # Exemple de structure en barres ou un code genre. 
        # On cherche la catégorie la plus haute. 
        max_g = 0
        for row in genre_div.select('.bars'):
            spans = row.find_all('span')
            if len(spans) == 2:
                g_name = spans[0].get_text(strip=True)
                g_vote = spans[1].get_text(strip=True)
                try:
                    g_vote = int(g_vote)
                    if g_vote > max_g:
                        max_g = g_vote
                        genre_plus_vote = g_name
                except ValueError:
                    pass
    
    data['genre_plus_vote'] = genre_plus_vote
    
    # 9) Longevite, Sillage (avec les votes)
    # Sur Fragrantica, c'est souvent un autre diagramme. 
    # Identifiants possibles : #diagram_longevity, #diagram_sillage ...
    
    longevite_votes = {}
    sillage_votes = {}
    
    # Longevity
    longevite_div = soup.select_one('#diagram_longevity')
    if longevite_div:
        # Structure possible : <div class="bars"><span>Very Weak</span><span>45</span></div>
        for bar in longevite_div.select('.bars'):
            spans = bar.find_all('span')
            if len(spans) == 2:
                label = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                longevite_votes[label] = value
    
    # Sillage
    sillage_div = soup.select_one('#diagram_sillage')
    if sillage_div:
        for bar in sillage_div.select('.bars'):
            spans = bar.find_all('span')
            if len(spans) == 2:
                label = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                sillage_votes[label] = value
    
    data['longevite_votes'] = longevite_votes
    data['sillage_votes'] = sillage_votes
    
    return data


if __name__ == "__main__":
    url_test = "https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html"
    infos = scrape_fragrantica_perfume(url_test)
    for k, v in infos.items():
        print(k, ":", v)


Status code: 403
Response snippet: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetic
nom_parfum : Black Phantom
marque : By Kilian
launch_year : None
perfume_rating : None
nb_votes : None
saison_plus_votee : None
parfumeur : None
main_accord : None
perfume_pyramid : {'top_notes': [], 'middle_notes': [], 'base_notes': []}
genre_plus_vote : None
longevite_votes : {}
sillage_votes : {}


C:\Users\tapri\AppData\Local\Temp\ipykernel_7360\3228269495.py:135: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  potential_perfumer = soup.find(text=re.compile(r'Perfumer:', re.IGNORECASE))


In [59]:
##On configure Chrome pour être en "headless" (sans interface)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
##On crée le driver en précisant les options headless
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


url = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"
html = get_html(driver, url)
soup = BeautifulSoup(html, 'html.parser')

title_el = soup.find('title')
launch_year = None

if title_el:
    title_text = title_el.get_text(strip=True)
    print("Titre complet :", title_text)

    # Méthode A : prendre les 4 derniers caractères (attention : suppose que l'année est exactement en fin de titre)
    possible_year = title_text[-4:]
    if possible_year.isdigit():
        launch_year = possible_year
    else:
        # Méthode B : un peu plus robuste, on cherche un pattern d'année 19xx ou 20xx dans tout le titre
        match = re.search(r'(19|20)\d{2}', title_text)
        if match:
            launch_year = match.group(0)

    print("Launch year:", launch_year)

Titre complet : Un instant…
Launch year: None


In [61]:
import cloudscraper
from bs4 import BeautifulSoup

url = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"

scraper = cloudscraper.create_scraper()  # crée un objet "requests" optimisé
response = scraper.get(url)

print("Status code:", response.status_code)
# Regardez un extrait du contenu pour vérifier si c’est la page correcte
print("Snippet:", response.text[:500])

# Ensuite, parsez avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
title_el = soup.find('title')
if title_el:
    print("Titre :", title_el.get_text(strip=True))


Status code: 403
Snippet: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetic
Titre : Just a moment...


In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

##On configure Chrome pour être en "headless" (sans interface)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
##On crée le driver en précisant les options headless
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


url = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"
driver.get(url)

# Laissez l'utilisateur (vous) voir le résultat, résoudre un éventuel captcha...
time.sleep(1)  # pour prendre le temps de vérifier/résoudre manuellement
print(driver.title)  # voir si vous avez enfin "Angels' Share..." ou "Just a moment..."
page_source = driver.page_source

driver.quit()


Un instant…


In [67]:
import undetected_chromedriver as uc
import time

def get_html_headless(url):
    """
    Lance Chrome en mode headless (via undetected-chromedriver),
    charge l'URL et retourne le HTML.
    """
    # 1) Options de Chrome
    options = uc.ChromeOptions()
    # '--headless=new' est conseillé depuis Chrome 109+ 
    # pour un headless moins détectable que l'ancien mode headless
    options.add_argument('--headless=new')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # Ajoutez d'autres arguments si nécessaire

    # 2) Instanciation d’un navigateur Chrome “furtif”
    driver = uc.Chrome(options=options)

    try:
        # 3) Ouverture de l’URL
        driver.get(url)

        # 4) Selon la complexité du site, un temps d’attente est parfois nécessaire
        time.sleep(5)  # Laissez le temps à la page de se charger, ajustez si besoin

        # 5) Récupération du titre pour vérifier si on est sur la vraie page
        print("Titre récupéré :", driver.title)

        # 6) Récupération du code source
        html_source = driver.page_source

    finally:
        # 7) Fermeture du navigateur
        driver.quit()

    return html_source

# ---------------------------
# EXEMPLE D'UTILISATION
# ---------------------------
if __name__ == "__main__":
    url_test = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"
    html_page = get_html_headless(url_test)
    print("Longueur du HTML :", len(html_page))


SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:4342
from session not created: This version of ChromeDriver only supports Chrome version 133
Current browser version is 132.0.6834.197
Stacktrace:
	GetHandleVerifier [0x01050283+25139]
	(No symbol) [0x00FDB234]
	(No symbol) [0x00EB04A3]
	(No symbol) [0x00EE96EB]
	(No symbol) [0x00EE8729]
	(No symbol) [0x00EDECDF]
	(No symbol) [0x00EDEB16]
	(No symbol) [0x00F28D18]
	(No symbol) [0x00F2872A]
	(No symbol) [0x00F1CD36]
	(No symbol) [0x00EEBD29]
	(No symbol) [0x00EED064]
	GetHandleVerifier [0x0135B143+3215603]
	GetHandleVerifier [0x013722BA+3310186]
	GetHandleVerifier [0x0136C4D2+3286146]
	GetHandleVerifier [0x010E9C80+654384]
	(No symbol) [0x00FE45BD]
	(No symbol) [0x00FE14A8]
	(No symbol) [0x00FE1647]
	(No symbol) [0x00FD3D20]
	BaseThreadInitThunk [0x76C97BA9+25]
	RtlInitializeExceptionChain [0x77D0C28B+107]
	RtlClearBits [0x77D0C20F+191]
